# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [33]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [34]:
#Copying csv path
file_to_load = "../Resources/clean_city_data.csv"

#Loading csv path onto dataframe using pandas read function
city_df = pd.read_csv(file_to_load)

city_df.head()

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bredasdorp,-34.53,20.04,57.20,71,0,9.17,ZA,1595866549
1,Albany,42.60,-73.97,91.00,39,86,1.99,US,1595866252
2,Avarua,-21.21,-159.78,69.80,88,57,5.82,CK,1595866552
3,Hithadhoo,-0.60,73.08,82.94,74,100,11.25,MV,1595866553
4,New Norfolk,-42.78,147.06,42.01,90,94,2.21,AU,1595866346


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [35]:
gmaps.configure(api_key=g_key)
humidity = city_df["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = city_df[["Latitude", "Longitude"]]

In [36]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [37]:
new_city_df = city_df.loc[(city_df["Max Temperature"] > 70) & (city_df["Max Temperature"] < 80) & 
            (city_df["Cloudiness"] == 0) & (city_df["Wind Speed"] < 10), :]

In [38]:
new_city_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
141,Luena,-11.78,19.92,73.11,29,0,4.52,AO,1595866736
210,Mwinilunga,-11.74,24.43,70.34,28,0,5.44,ZM,1595866825
216,Kirando,-7.42,30.60,73.35,55,0,6.31,TZ,1595866833
235,Opuwo,-18.06,13.84,77.34,18,0,8.88,NaN,1595866866
258,Colomi,-17.35,-65.87,71.60,17,0,2.66,BO,1595866921
307,Arkhipo-Osipovka,44.37,38.52,76.98,73,0,2.57,RU,1595866991
358,Athabasca,54.72,-113.29,71.01,66,0,3.13,CA,1595866824
382,Xinzhi,36.50,111.70,75.83,70,0,6.42,CN,1595867112
383,Sumbe,-11.21,13.84,73.56,73,0,7.61,AO,1595867113
415,Ponta do Sol,32.67,-17.10,75.00,65,0,4.88,PT,1595866925


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [40]:
#Empty list to store Hotel data.
Hotel = []

#Get latitude and longitude from dataframe (row/column) using 'iloc'.
for i in range(0,len(new_city_df)):
   
    latitude = new_city_df.iloc[i,1]
    longitude = new_city_df.iloc[i,2]
    
    #Setting necessary parameters to search for.
    params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
    }
    
    #Assigning location parameter to latitude, longitude.
    params["location"] = f"{latitude},{longitude}"
    
    #Requesting Places API to retreive Hotel info.
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    places_requested = requests.get(url, params=params).json()
    try:
        Hotel.append(places_requested['results'][0]['name'])
    
    except IndexError:
        Hotel.append(np.nan)

In [41]:
new_city_df["Hotel Name"] = Hotel
hotel_df = new_city_df.dropna(how='any')

hotel_df.head()

/Users/adithyavenkatesh/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
141,Luena,-11.78,19.92,73.11,29,0,4.52,AO,1595866736,Hotel Kandamba
210,Mwinilunga,-11.74,24.43,70.34,28,0,5.44,ZM,1595866825,Muzina Guest House
216,Kirando,-7.42,30.60,73.35,55,0,6.31,TZ,1595866833,Lakeshore Lodge
258,Colomi,-17.35,-65.87,71.60,17,0,2.66,BO,1595866921,Alojamiento Hinojosa
307,Arkhipo-Osipovka,44.37,38.52,76.98,73,0,2.57,RU,1595866991,Otel' Roza Vetrov


In [42]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [43]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))